<a href="https://colab.research.google.com/github/psukphranee/Machine-Learning-Projects/blob/master/Keras_Multi_input_one_output__multi_input_multi_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras

In [33]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

In [34]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [69]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


In [51]:
display(scaler.get_params)

<bound method BaseEstimator.get_params of StandardScaler()>

In [60]:
#create model

input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

concat = keras.layers.concatenate([input_A, hidden2])

output = keras.layers.Dense(1, name="main_output")(concat)

model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [72]:

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [70]:
X_train_A, X_train_B = X_train[:,:5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:,:5], X_valid[:, 2:]

X_test_A, X_test_B = X_test[:,:5], X_test[:, 2:]

X_new_A = X_test_A[:3]
X_new_B = X_test_B[:3]

In [56]:
X_train_B[:3]

array([[-0.94338545,  0.01485314,  0.02073335, -0.57291624,  0.92926047,
        -1.42215523],
       [ 0.40547793, -0.23327682,  1.8614649 ,  0.20516532, -0.91654738,
         1.09666969],
       [ 0.81808819,  1.05678372, -0.08786707, -0.29983271,  1.30872858,
        -1.697027  ]])

In [73]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.9320 - val_loss: 1.1291
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7295 - val_loss: 0.6691
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6232 - val_loss: 0.5800
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5684 - val_loss: 0.5392
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5324 - val_loss: 0.5093
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5062 - val_loss: 0.4867
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4872 - val_loss: 0.4671
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4736 - val_loss: 0.4517
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4631 - val_loss: 0.4357
Epoch 10/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4554 - val_loss: 0.4250

In [76]:

aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model2 = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])
model2.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [77]:
history2 = model2.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.8918 - main_output_loss: 0.7922 - aux_output_loss: 1.7888 - val_loss: 2.3906 - val_main_output_loss: 2.3913 - val_aux_output_loss: 2.3845
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5219 - main_output_loss: 0.4741 - aux_output_loss: 0.9526 - val_loss: 8.0645 - val_main_output_loss: 8.4291 - val_aux_output_loss: 4.7833
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5462 - main_output_loss: 0.5155 - aux_output_loss: 0.8224 - val_loss: 0.8844 - val_main_output_loss: 0.8713 - val_aux_output_loss: 1.0017
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4494 - main_output_loss: 0.4211 - aux_output_loss: 0.7045 - val_loss: 0.4483 - val_main_output_loss: 0.4247 - val_aux_output_loss: 0.6609
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4309 - main_output_loss: 0.4069 - aux_output_loss: 0.6471 - val_loss

In [78]:
total_loss, main_loss, aux_loss = model2.evaluate(
    [X_test_A, X_test_B], [y_test, y_test]
)

162/162 [==============================] - 0s 1ms/step - loss: 0.3563 - main_output_loss: 0.3431 - aux_output_loss: 0.4745
